In [2]:
import chess
import pandas as pd
from datasets import load_dataset
import numpy as np
import re

/data/evan/miniconda3/envs/chess/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
data = load_dataset("evanfrick/ccc", num_proc=32)

Resolving data files: 100%|██████████| 723/723 [00:02<00:00, 337.14it/s]


In [4]:
ccc = pd.DataFrame(data['train'])

In [5]:
ccc

,id,game,result
0,d617a15a3fb06d417e2304f3f7779df111d653387f637d...,"[{'move_num': 4, 'turn': '<w>', 'seq_in': '<w>...",1-0
1,8d4c06f25249b703d29c2058402c1b13fffc8351a04d0c...,"[{'move_num': 4, 'turn': '<w>', 'seq_in': '<w>...",1/2-1/2
2,56c5bae531f0d62007d6b3486a7c61d4dcc56a114cc90c...,"[{'move_num': 4, 'turn': '<w>', 'seq_in': '<w>...",1-0
3,8c19a4d5494e0670d73180723ff1b831d9b324a3135265...,"[{'move_num': 4, 'turn': '<w>', 'seq_in': '<w>...",0-1
4,668f30a44288ccaf0b811201bb062cc12659c118b36ab7...,"[{'move_num': 4, 'turn': '<w>', 'seq_in': '<w>...",1-0
...,...,...,...
168939,ffff45a3e933ed6b8ebb2de013edbada4b04959a6157af...,"[{'move_num': 7, 'turn': '<w>', 'seq_in': '<w>...",1/2-1/2
168940,568b8af29650da5fb25a3a9e867f1acbf71ed73c4b8515...,"[{'move_num': 7, 'turn': '<b>', 'seq_in': '<w>...",1/2-1/2
168941,33e657d0d0d68b29a322279c7d364bfa11096c8c8dc271...,"[{'move_num': 7, 'turn': '<b>', 'seq_in': '<w>...",1/2-1/2
168942,11b1874de03eff3866aa7fac0e92729452bfe97f466b5f...,"[{'move_num': 6, 'turn': '<b>', 'seq_in': '<w>...",1/2-1/2


In [6]:
ccc_explode = ccc.explode("game")

In [7]:
ccc_explode

,id,game,result
0,d617a15a3fb06d417e2304f3f7779df111d653387f637d...,"{'move_num': 4, 'turn': '<w>', 'seq_in': '<w>e...",1-0
0,d617a15a3fb06d417e2304f3f7779df111d653387f637d...,"{'move_num': 4, 'turn': '<b>', 'seq_in': '<w>e...",1-0
0,d617a15a3fb06d417e2304f3f7779df111d653387f637d...,"{'move_num': 5, 'turn': '<w>', 'seq_in': '<w>e...",1-0
0,d617a15a3fb06d417e2304f3f7779df111d653387f637d...,"{'move_num': 5, 'turn': '<b>', 'seq_in': '<w>e...",1-0
0,d617a15a3fb06d417e2304f3f7779df111d653387f637d...,"{'move_num': 6, 'turn': '<w>', 'seq_in': '<w>e...",1-0
...,...,...,...
168943,e7b121e90c2bf8a4582b23e7a948a310bc33e158a7b463...,"{'move_num': 48, 'turn': '<b>', 'seq_in': '<w>...",0-1
168943,e7b121e90c2bf8a4582b23e7a948a310bc33e158a7b463...,"{'move_num': 49, 'turn': '<w>', 'seq_in': '<w>...",0-1
168943,e7b121e90c2bf8a4582b23e7a948a310bc33e158a7b463...,"{'move_num': 49, 'turn': '<b>', 'seq_in': '<w>...",0-1
168943,e7b121e90c2bf8a4582b23e7a948a310bc33e158a7b463...,"{'move_num': 50, 'turn': '<w>', 'seq_in': '<w>...",0-1


In [8]:
moves = ccc_explode.game.reset_index(drop=True).dropna()

In [9]:
lines = ccc_explode.drop('game', axis=1).reset_index(drop=True).join(pd.DataFrame.from_dict(moves.to_dict(), orient='index'))

In [10]:
lines

,id,result,move_num,turn,seq_in,seq_out,eval
0,d617a15a3fb06d417e2304f3f7779df111d653387f637d...,1-0,4.0,<w>,<w>e2-e4<b>c7-c5<w>Ng1-f3<b>d7-d6<w>d2-d4<b>c5...,Qd1xd4,0.17
1,d617a15a3fb06d417e2304f3f7779df111d653387f637d...,1-0,4.0,<b>,<w>e2-e4<b>c7-c5<w>Ng1-f3<b>d7-d6<w>d2-d4<b>c5...,Nb8-c6<w>Qd4-e3<b>Ng8-f6<w>Nb1-c3<b>Nf6-g4<w>Q...,0.27
2,d617a15a3fb06d417e2304f3f7779df111d653387f637d...,1-0,5.0,<w>,<w>e2-e4<b>c7-c5<w>Ng1-f3<b>d7-d6<w>d2-d4<b>c5...,Qd4-d3,0.09
3,d617a15a3fb06d417e2304f3f7779df111d653387f637d...,1-0,5.0,<b>,<w>e2-e4<b>c7-c5<w>Ng1-f3<b>d7-d6<w>d2-d4<b>c5...,g7-g6<w>Bf1-e2<b>Ng8-f6<w>O-O<b>Bf8-g7<w>Nb1-c...,0.26
4,d617a15a3fb06d417e2304f3f7779df111d653387f637d...,1-0,6.0,<w>,<w>e2-e4<b>c7-c5<w>Ng1-f3<b>d7-d6<w>d2-d4<b>c5...,Bc1-e3,0.09
...,...,...,...,...,...,...,...
25034615,e7b121e90c2bf8a4582b23e7a948a310bc33e158a7b463...,0-1,48.0,<b>,<w>e2-e4<b>e7-e5<w>f2-f4<b>e5xf4<w>Bf1-c4<b>Qd...,Qf1-f2+<w>Kd2-c3<b>Qf2-b2+<w>Kc3xd3<b>Rh8-h3#,-M5
25034616,e7b121e90c2bf8a4582b23e7a948a310bc33e158a7b463...,0-1,49.0,<w>,<w>e2-e4<b>e7-e5<w>f2-f4<b>e5xf4<w>Bf1-c4<b>Qd...,Kd2-c3<b>Qf2-b2+<w>Kc3xd3<b>Rh8-h3#,-M4
25034617,e7b121e90c2bf8a4582b23e7a948a310bc33e158a7b463...,0-1,49.0,<b>,<w>e2-e4<b>e7-e5<w>f2-f4<b>e5xf4<w>Bf1-c4<b>Qd...,Qf2-b2+<w>Kc3xd3<b>Rh8-h3#,-M3
25034618,e7b121e90c2bf8a4582b23e7a948a310bc33e158a7b463...,0-1,50.0,<w>,<w>e2-e4<b>e7-e5<w>f2-f4<b>e5xf4<w>Bf1-c4<b>Qd...,Kc3xd3<b>Rh8-h3#,-M2


In [11]:
lines.sort_values("move_num", inplace=True)

In [12]:
def average_eval(s):
    try:
        return s[:2].mean()
    except Exception as e:
        return np.NaN
    
def scale_eval(ev):
    if ev is None:
        return np.NaN
    elif type(ev) is float:
        return ev
    elif "-M" in ev:
        return 0
    elif "M" in ev:
        return 1
    else:
        ev = re.findall(r"[-+]?(?:\d*\.*\d+)", ev)[0]
        ev = float(ev)
        return 1 / (1 + np.exp(-ev))

In [13]:
lines['scaled_eval'] = lines['eval'].map(scale_eval)

/tmp/ipykernel_1654063/2035028842.py:19: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-ev))


In [14]:
starting = lines.groupby("id").agg({'result': 'first', 'move_num': 'first', 'scaled_eval': average_eval})
starting

,result,move_num,scaled_eval
id,,,
0000597279c47403d381b38e854a7ea183c13dc026cb0c22416e925b3275de85,1-0,9.0,0.746171
000084372c0fa7fbadf7f97f073b4237ac248fad3230734a6015f2a209d4cb4a,1/2-1/2,8.0,0.513716
0000c1c3a147bf0cf478060892517b7f869ece1a5dc66e0964af9f805eca416b,1-0,1.0,0.565719
0001f9013be729f4f4598d50fc7e120298504549d11157051db1e818bb8855f5,1-0,9.0,0.783317
0002c0642618b2f1b48834d05ddf15906d7b1e0cce0a8918b74d147622a48257,1/2-1/2,9.0,0.728670
...,...,...,...
fffe1bb875b62d486780dbf519eabcd0e42f4ce04d6c86619a797eaf2bef1e1b,1-0,7.0,0.769982
fffec6bf244b54fe34017be51ca651ca94215c0771a32e295b31413d762cc78f,1/2-1/2,9.0,0.533673
fffec92846d7855d4c3aed2d1d1ad0e7c7b7f6830592558c95a62dc40022e431,1/2-1/2,7.0,0.537370


In [15]:
starting.dropna(inplace=True)

In [16]:
starting

,result,move_num,scaled_eval
id,,,
0000597279c47403d381b38e854a7ea183c13dc026cb0c22416e925b3275de85,1-0,9.0,0.746171
000084372c0fa7fbadf7f97f073b4237ac248fad3230734a6015f2a209d4cb4a,1/2-1/2,8.0,0.513716
0000c1c3a147bf0cf478060892517b7f869ece1a5dc66e0964af9f805eca416b,1-0,1.0,0.565719
0001f9013be729f4f4598d50fc7e120298504549d11157051db1e818bb8855f5,1-0,9.0,0.783317
0002c0642618b2f1b48834d05ddf15906d7b1e0cce0a8918b74d147622a48257,1/2-1/2,9.0,0.728670
...,...,...,...
fffe1bb875b62d486780dbf519eabcd0e42f4ce04d6c86619a797eaf2bef1e1b,1-0,7.0,0.769982
fffec6bf244b54fe34017be51ca651ca94215c0771a32e295b31413d762cc78f,1/2-1/2,9.0,0.533673
fffec92846d7855d4c3aed2d1d1ad0e7c7b7f6830592558c95a62dc40022e431,1/2-1/2,7.0,0.537370


In [17]:
starting['temp'] = list(zip(starting['result'], starting['scaled_eval']))

In [18]:
starting

,result,move_num,scaled_eval,temp
id,,,,
0000597279c47403d381b38e854a7ea183c13dc026cb0c22416e925b3275de85,1-0,9.0,0.746171,"(1-0, 0.7461709834378405)"
000084372c0fa7fbadf7f97f073b4237ac248fad3230734a6015f2a209d4cb4a,1/2-1/2,8.0,0.513716,"(1/2-1/2, 0.5137155892318921)"
0000c1c3a147bf0cf478060892517b7f869ece1a5dc66e0964af9f805eca416b,1-0,1.0,0.565719,"(1-0, 0.5657191874165081)"
0001f9013be729f4f4598d50fc7e120298504549d11157051db1e818bb8855f5,1-0,9.0,0.783317,"(1-0, 0.7833168071751857)"
0002c0642618b2f1b48834d05ddf15906d7b1e0cce0a8918b74d147622a48257,1/2-1/2,9.0,0.728670,"(1/2-1/2, 0.7286697340028847)"
...,...,...,...,...
fffe1bb875b62d486780dbf519eabcd0e42f4ce04d6c86619a797eaf2bef1e1b,1-0,7.0,0.769982,"(1-0, 0.769982347118076)"
fffec6bf244b54fe34017be51ca651ca94215c0771a32e295b31413d762cc78f,1/2-1/2,9.0,0.533673,"(1/2-1/2, 0.5336734791493356)"
fffec92846d7855d4c3aed2d1d1ad0e7c7b7f6830592558c95a62dc40022e431,1/2-1/2,7.0,0.537370,"(1/2-1/2, 0.5373703561324727)"


In [19]:
choice = ['<b>', '<w>']
def convert_result_to_number(result):
    if result == '1-0':
        return 1
    elif result == '0-1':
        return 0
    elif result == '1/2-1/2':
        return 0.5
    else:
        return 0.5
    
#starting['result_n'] = starting.result.map(convert_result_to_number)
starting['pick'] = starting['temp'].map(lambda t: choice[int(convert_result_to_number(t[0]) >= np.round(t[1], 1))])

In [20]:
starting

,result,move_num,scaled_eval,temp,pick
id,,,,,
0000597279c47403d381b38e854a7ea183c13dc026cb0c22416e925b3275de85,1-0,9.0,0.746171,"(1-0, 0.7461709834378405)",<w>
000084372c0fa7fbadf7f97f073b4237ac248fad3230734a6015f2a209d4cb4a,1/2-1/2,8.0,0.513716,"(1/2-1/2, 0.5137155892318921)",<w>
0000c1c3a147bf0cf478060892517b7f869ece1a5dc66e0964af9f805eca416b,1-0,1.0,0.565719,"(1-0, 0.5657191874165081)",<w>
0001f9013be729f4f4598d50fc7e120298504549d11157051db1e818bb8855f5,1-0,9.0,0.783317,"(1-0, 0.7833168071751857)",<w>
0002c0642618b2f1b48834d05ddf15906d7b1e0cce0a8918b74d147622a48257,1/2-1/2,9.0,0.728670,"(1/2-1/2, 0.7286697340028847)",<b>
...,...,...,...,...,...
fffe1bb875b62d486780dbf519eabcd0e42f4ce04d6c86619a797eaf2bef1e1b,1-0,7.0,0.769982,"(1-0, 0.769982347118076)",<w>
fffec6bf244b54fe34017be51ca651ca94215c0771a32e295b31413d762cc78f,1/2-1/2,9.0,0.533673,"(1/2-1/2, 0.5336734791493356)",<w>
fffec92846d7855d4c3aed2d1d1ad0e7c7b7f6830592558c95a62dc40022e431,1/2-1/2,7.0,0.537370,"(1/2-1/2, 0.5373703561324727)",<w>


In [21]:
starting.pick.value_counts()

pick
<b>    86060
<w>    77432
Name: count, dtype: int64

In [22]:
pick = starting.pick

In [23]:
lines = lines.join(pick, on='id', how='inner')

In [24]:
out_ccc = lines[lines['turn'] == lines['pick']]
out_ccc

,id,result,move_num,turn,seq_in,seq_out,eval,scaled_eval,pick
9673948,d75016f689d78bfb8ea51330ed006446acf187aa2b5a08...,1/2-1/2,1.0,<b>,<w>e2-e4<b>,e7-e5<w>Ne1-d3<b>f7-f6<w>f2-f4<b>c7-c5<w>c2-c4...,0.24,0.559714,<b>
9673950,d75016f689d78bfb8ea51330ed006446acf187aa2b5a08...,1/2-1/2,2.0,<b>,<w>e2-e4<b>e7-e5<w>Ne1-d3<b>,f7-f6<w>f2-f4<b>c7-c5<w>c2-c4<b>b7-b6<w>Nh1-g3...,0.40,0.598688,<b>
9673952,d75016f689d78bfb8ea51330ed006446acf187aa2b5a08...,1/2-1/2,3.0,<b>,<w>e2-e4<b>e7-e5<w>Ne1-d3<b>f7-f6<w>f2-f4<b>,c7-c5<w>c2-c4<b>e5xf4<w>Nd3xf4<b>Bd8-c7<w>Nf4-...,0.58,0.641067,<b>
9673954,d75016f689d78bfb8ea51330ed006446acf187aa2b5a08...,1/2-1/2,4.0,<b>,<w>e2-e4<b>e7-e5<w>Ne1-d3<b>f7-f6<w>f2-f4<b>c7...,e5xf4<w>Nd3xf4<b>Nh8-g6<w>Nf4-d5<b>Bd8-c7<w>Nd...,0.56,0.636453,<b>
9673956,d75016f689d78bfb8ea51330ed006446acf187aa2b5a08...,1/2-1/2,5.0,<b>,<w>e2-e4<b>e7-e5<w>Ne1-d3<b>f7-f6<w>f2-f4<b>c7...,Nh8-g6<w>Nf4-d5,0.31,0.576885,<b>
...,...,...,...,...,...,...,...,...,...
14306250,a3d432257a5cbb13f6d731a4ba623d20cb572d6eb1ae09...,1/2-1/2,185.0,<w>,<w>d2-d4<b>f7-f5<w>Nb1-c3<b>Ng8-f6<w>Bc1-g5<b>...,Rh1-h7+,"0.00, Draw by fifty moves rule",0.500000,<w>
3179550,98a5f62d8f6314d4226c86209222fe5a55c8430e895772...,1/2-1/2,182.0,<w>,<w>Ng1-f3<b>c7-c5<w>e2-e4<b>Nb8-c6<w>Nb1-c3<b>...,Rd1-e1+<b>Rd2-e2<w>Re1-f1<b>Bf4-e5<w>Rf1-f3+<b...,"-4.95, Draw by adjudication: SyzygyTB",0.007034,<w>
12982973,58f8343681ffefe74d5437924307f2d748b8348cff6973...,1/2-1/2,218.0,<w>,<w>Ng1-f3<b>Ng8-f6<w>d2-d4<b>e7-e6<w>c2-c4<b>b...,Qe4-e3<b>Rf7-b7<w>Qe3-c1<b>Kf8-f7<w>Bc3-b2,0.00,0.500000,<w>
12982975,58f8343681ffefe74d5437924307f2d748b8348cff6973...,1/2-1/2,219.0,<w>,<w>Ng1-f3<b>Ng8-f6<w>d2-d4<b>e7-e6<w>c2-c4<b>b...,Bc2-d3<b>Nc7-e8<w>Kf1-e1,0.00,0.500000,<w>


In [25]:
out_ccc.pick.value_counts()

pick
<b>    6512957
<w>    5715274
Name: count, dtype: int64

In [26]:
out_ccc['seq_out_tokens'] = out_ccc.seq_out.str.count("<")

/tmp/ipykernel_1654063/2883070674.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  out_ccc['seq_out_tokens'] = out_ccc.seq_out.str.count("<")


In [27]:
out_ccc

,id,result,move_num,turn,seq_in,seq_out,eval,scaled_eval,pick,seq_out_tokens
9673948,d75016f689d78bfb8ea51330ed006446acf187aa2b5a08...,1/2-1/2,1.0,<b>,<w>e2-e4<b>,e7-e5<w>Ne1-d3<b>f7-f6<w>f2-f4<b>c7-c5<w>c2-c4...,0.24,0.559714,<b>,31.0
9673950,d75016f689d78bfb8ea51330ed006446acf187aa2b5a08...,1/2-1/2,2.0,<b>,<w>e2-e4<b>e7-e5<w>Ne1-d3<b>,f7-f6<w>f2-f4<b>c7-c5<w>c2-c4<b>b7-b6<w>Nh1-g3...,0.40,0.598688,<b>,30.0
9673952,d75016f689d78bfb8ea51330ed006446acf187aa2b5a08...,1/2-1/2,3.0,<b>,<w>e2-e4<b>e7-e5<w>Ne1-d3<b>f7-f6<w>f2-f4<b>,c7-c5<w>c2-c4<b>e5xf4<w>Nd3xf4<b>Bd8-c7<w>Nf4-...,0.58,0.641067,<b>,29.0
9673954,d75016f689d78bfb8ea51330ed006446acf187aa2b5a08...,1/2-1/2,4.0,<b>,<w>e2-e4<b>e7-e5<w>Ne1-d3<b>f7-f6<w>f2-f4<b>c7...,e5xf4<w>Nd3xf4<b>Nh8-g6<w>Nf4-d5<b>Bd8-c7<w>Nd...,0.56,0.636453,<b>,35.0
9673956,d75016f689d78bfb8ea51330ed006446acf187aa2b5a08...,1/2-1/2,5.0,<b>,<w>e2-e4<b>e7-e5<w>Ne1-d3<b>f7-f6<w>f2-f4<b>c7...,Nh8-g6<w>Nf4-d5,0.31,0.576885,<b>,1.0
...,...,...,...,...,...,...,...,...,...,...
14306250,a3d432257a5cbb13f6d731a4ba623d20cb572d6eb1ae09...,1/2-1/2,185.0,<w>,<w>d2-d4<b>f7-f5<w>Nb1-c3<b>Ng8-f6<w>Bc1-g5<b>...,Rh1-h7+,"0.00, Draw by fifty moves rule",0.500000,<w>,0.0
3179550,98a5f62d8f6314d4226c86209222fe5a55c8430e895772...,1/2-1/2,182.0,<w>,<w>Ng1-f3<b>c7-c5<w>e2-e4<b>Nb8-c6<w>Nb1-c3<b>...,Rd1-e1+<b>Rd2-e2<w>Re1-f1<b>Bf4-e5<w>Rf1-f3+<b...,"-4.95, Draw by adjudication: SyzygyTB",0.007034,<w>,15.0
12982973,58f8343681ffefe74d5437924307f2d748b8348cff6973...,1/2-1/2,218.0,<w>,<w>Ng1-f3<b>Ng8-f6<w>d2-d4<b>e7-e6<w>c2-c4<b>b...,Qe4-e3<b>Rf7-b7<w>Qe3-c1<b>Kf8-f7<w>Bc3-b2,0.00,0.500000,<w>,4.0
12982975,58f8343681ffefe74d5437924307f2d748b8348cff6973...,1/2-1/2,219.0,<w>,<w>Ng1-f3<b>Ng8-f6<w>d2-d4<b>e7-e6<w>c2-c4<b>b...,Bc2-d3<b>Nc7-e8<w>Kf1-e1,0.00,0.500000,<w>,2.0


In [28]:
out_ccc['ply'] = (out_ccc['move_num'] - 1) * 2 + (out_ccc['turn'] == "<b>")

/tmp/ipykernel_1654063/3425384982.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  out_ccc['ply'] = (out_ccc['move_num'] - 1) * 2 + (out_ccc['turn'] == "<b>")


In [29]:
out_ccc.sort_values(['ply', 'seq_out_tokens'], inplace=True)

/tmp/ipykernel_1654063/194384951.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  out_ccc.sort_values(['ply', 'seq_out_tokens'], inplace=True)


In [30]:
out_ccc

,id,result,move_num,turn,seq_in,seq_out,eval,scaled_eval,pick,seq_out_tokens,ply
4377768,ee2c1b6d91dbf60f232bf44dcf47f485c03c4248b1e4d5...,1/2-1/2,1.0,<w>,<w>,d2-d4,0.23,0.557248,<w>,0.0,0.0
4106199,60f83035b03181f85f4a00ab9f2e5326f01016f7261b59...,1-0,1.0,<w>,<w>,e2-e4,0.24,0.559714,<w>,0.0,0.0
8404497,c48bd6531b850aaebb3bd8b01cf1d81b43690aff392fe8...,1-0,1.0,<w>,<w>,d2-d4,0.33,0.581759,<w>,0.0,0.0
14503979,ba60c6d84da8d6159e3c7b2f94d3fbbc146c972447be2a...,1/2-1/2,1.0,<w>,<w>,d2-d4,0.11,0.527472,<w>,0.0,0.0
1099654,336a1f88c2f724306efc1c39511a9b1be1e29a0be61e34...,1-0,1.0,<w>,<w>,e2-e4,0.22,0.554779,<w>,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
9275467,9489d9e08817ec0d1cdf7f66dafa2097125bc39d7b0058...,1/2-1/2,497.0,<b>,<w>e2-e4<b>e7-e5<w>Ng1-f3<b>Nb8-c6<w>Bf1-b5<b>...,Kd7-e6<w>Bf6-e5<b>Be4-h1<w>Kd4-e3<b>Ke6-d5<w>K...,-0.00,0.500000,<b>,6.0,993.0
9275469,9489d9e08817ec0d1cdf7f66dafa2097125bc39d7b0058...,1/2-1/2,498.0,<b>,<w>e2-e4<b>e7-e5<w>Ng1-f3<b>Nb8-c6<w>Bf1-b5<b>...,Be4-h1<w>Kd4-e3<b>Bh1-e4<w>Ke3-d4<b>Be4-h1<w>B...,-0.00,0.500000,<b>,6.0,995.0
9275471,9489d9e08817ec0d1cdf7f66dafa2097125bc39d7b0058...,1/2-1/2,499.0,<b>,<w>e2-e4<b>e7-e5<w>Ng1-f3<b>Nb8-c6<w>Bf1-b5<b>...,Ke6-f7<w>Kc4-d4<b>Kf7-e6<w>Bh8-e5<b>Bh1-d5<w>B...,-0.01,0.497500,<b>,6.0,997.0
9275473,9489d9e08817ec0d1cdf7f66dafa2097125bc39d7b0058...,1/2-1/2,500.0,<b>,<w>e2-e4<b>e7-e5<w>Ng1-f3<b>Nb8-c6<w>Bf1-b5<b>...,Kf7-g8<w>Bh8-e5<b>Kg8-f7<w>Kb5-c5<b>Bh1-g2<w>B...,-0.01,0.497500,<b>,6.0,999.0


In [31]:
out_ccc['total_tokens'] = out_ccc['ply'] + out_ccc['seq_out_tokens'] + 1

/tmp/ipykernel_1654063/3655561339.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  out_ccc['total_tokens'] = out_ccc['ply'] + out_ccc['seq_out_tokens'] + 1


In [32]:
(out_ccc['ply'] * 2 < 768).sum()

12149106

In [33]:
!pwd

/data/evan/CS285_Final_Project/data


In [38]:
out_ccc = out_ccc[~out_ccc['seq_out'].isna()]

In [39]:
out_ccc[out_ccc['ply'] * 2 < 767][['id', 'seq_in', 'seq_out', 'scaled_eval', 'ply', 'seq_out_tokens', 'total_tokens', 'turn']].to_parquet("ccc_processed.parquet", index=False)


In [40]:
out_ccc.describe()

,move_num,scaled_eval,seq_out_tokens,ply
count,1.222823e+07,1.212282e+07,1.212282e+07,1.222823e+07
mean,5.125216e+01,6.258492e-01,2.476516e+01,1.010369e+02
std,3.644817e+01,2.554504e-01,1.626795e+01,7.292603e+01
min,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.500000e+01,5.000000e-01,1.100000e+01,4.800000e+01
50%,4.400000e+01,5.670929e-01,2.500000e+01,8.700000e+01
75%,6.800000e+01,8.249137e-01,3.600000e+01,1.340000e+02
max,5.010000e+02,1.000000e+00,2.430000e+02,1.001000e+03


In [1]:
out_ccc

NameError: name 'out_ccc' is not defined